In [26]:
using LinearAlgebra

M = [2 3+im 1 -3; 3-im 1 -5-(2*im) (2*im); 1 -5-(2*im) 2 -2; -3 -(2*im) -2 -3]

4×4 Matrix{Complex{Int64}}:
  2+0im   3+1im   1+0im  -3+0im
  3-1im   1+0im  -5-2im   0+2im
  1+0im  -5-2im   2+0im  -2+0im
 -3+0im   0-2im  -2+0im  -3+0im

In [27]:
# Complex conjugate transpose of M

M_conj_trans = copy(M)

for i in 1:4
    for j in 1:4
        M_conj_trans[i,j] = conj(M[j,i])
    end
end

M_conj_trans

4×4 Matrix{Complex{Int64}}:
  2+0im   3+1im   1+0im  -3+0im
  3-1im   1+0im  -5+2im   0+2im
  1+0im  -5+2im   2+0im  -2+0im
 -3+0im   0-2im  -2+0im  -3+0im

In [28]:
# Check if M is Hermitian from its eigenvalues

eigvals(M)

4-element Vector{ComplexF64}:
 -5.359981380223615 - 0.8881457409055221im
 -4.325505357273712 - 0.741443276571322im
  4.361436457983973 - 0.03282459605358637im
  7.324050279513358 + 1.6624136135304348im

In [29]:
# Check if M is Hermitian

M == M_conj_trans

false

In [30]:
# Check if M is skew Hermitian

M_conj_trans == -M

false

In [ ]:
# Check if M is unitary

M_conj_trans * M == I(4)

false

### Verdict
M is not Hermitian, $M^H \neq M$, the eigenvalues of $M$ are also complex.

M is also not skew-Hermitian, $M^H \neq -M$.

M is also not unitary, $M^H M \neq 1$.